<h2>Install & import required packages<h2>

In [1]:
!pip install transformers
!pip install transformers torch
!pip install datasets
!pip install evaluate
!pip install rouge_score

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import evaluate
import nltk

from PIL import Image
from transformers import AutoProcessor, AutoModelForSeq2SeqLM, AutoModelForVision2Seq, AutoTokenizer, AutoModelForCausalLM, pipeline, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from tqdm import tqdm
from datasets import load_dataset
from nltk.tokenize import SyllableTokenizer
from nltk import word_tokenize

<h2>Load models<h2>

In [2]:
# img2word model (kosmos) from https://huggingface.co/microsoft/led-base-16384
kosmos_name = "microsoft/kosmos-2-patch14-224"                    
kosmos_model = AutoModelForVision2Seq.from_pretrained(kosmos_name)
kosmos_processor = AutoProcessor.from_pretrained(kosmos_name)

model.safetensors:   0%|          | 0.00/6.66G [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
### load word to haiku model
model_name = "fabianmmueller/deep-haiku-gpt-2"

syllable_tokenizer = SyllableTokenizer()
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False, return_tensors = "pt")

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

**Image2Word**

In [6]:
prompt = "<grounding>An image of"

# User inputted image
image = Image.open("data/pig.jpeg")

inputs = kosmos_processor(text=prompt, images=image, return_tensors="pt")

generated_ids = kosmos_model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=128,
)
generated_text = kosmos_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Specify `cleanup_and_extract=False` in order to see the raw model generation.
processed_text = kosmos_processor.post_process_generation(generated_text, cleanup_and_extract=False)

# print(processed_text)
# `<grounding> An image of<phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> warming himself by<phrase> a fire</phrase><object><patch_index_0005><patch_index_0911></object>.`

# By default, the generated  text is cleanup and the entities are extracted.
processed_text, entities = kosmos_processor.post_process_generation(generated_text)

kosmos_output = entities[0][0]
print(kosmos_output)

a piglet in a field


**Word2Haiku**

In [8]:
### Generating sample outputs with the pretrained model
prompt = "( snowman = "
prompt = "( " + kosmos_output + " = "

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

generator = pipeline('text-generation', model = model_name)

result = generator(prompt)

### Cleaning results from pretrained model & measure perplexity in comparison to gpt2
result_text = result[0]['generated_text']
start = result_text.find("=")
end = result_text.find(")")
cleaned_result = result_text[start+1:end].strip()
print(cleaned_result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


urn of snow. / A piglet. / In a lime box.
